In [53]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

import numpy as np
import pandas as pd
import seaborn as sns

import pandas as pd
from pandas import DataFrame

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors 

In [54]:
df_data = pd.read_csv('data.csv', index_col=0).drop(columns=['Unnamed: 0'])
df_bac = pd.read_csv('bacterial_descriptors.csv', index_col=0)
df_drug = pd.read_csv('drug_descriptors.csv', index_col=0)

In [55]:
df_data.head(1)

,Bacteria,NP_Synthesis,Drug,Drug_class_drug_bank,Drug_dose,NP_concentration,NP size_min,NP size_max,NP size_avg,shape,method,ZOI_drug,ZOI_NP,ZOI_drug_NP,fold_increase_in_antibacterial_activity (%),MDR_check
0,Escherichia coli,chem_synthesis_reduction_by_D_maltose,Amoxicillin,Lactams,NaN,NaN,4.0,10.0,8.0,spherical,MIC,32+,6.3,32+,NaN,1


In [56]:
df_bac.head(1)

,Bacteria,kingdom,subkingdom,clade,phylum,class,order,family,genus,species,gram,"min_Incub_period, h","avg_Incub_period, h","max_Incub_period, h","growth_temp, C",biosafety_level,isolated_from
Tax_id,,,,,,,,,,,,,,,,,
['470'],Acinetobacter baumannii,Bacteria,NaN,NaN,Pseudomonadota,Gammaproteobacteria,Moraxellales,Moraxellaceae,Acinetobacter,Acinetobacter calcoaceticus/baumannii complex,n,24.0,36.0,48.0,37,2,urine


In [57]:
df_drug.head(1)

,drug,chemID,prefered_name,smiles
0,Amoxicillin,CHEMBL1082,AMOXICILLIN,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccc(O)cc3)...


# Работаем с драгами

In [58]:
df_drug.drop(columns=['prefered_name', 'chemID'], inplace=True)
df_drug.head(1)

,drug,smiles
0,Amoxicillin,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccc(O)cc3)...


In [59]:
drugs_in_drugs = set(df_drug.drug.unique())
drugs_in_data = set(df_data.Drug.unique())
drugs_in_data.difference(drugs_in_drugs)

{'Neomycin', nan}

Неомицин добавим вручную, nan будем считать наночастицами без модификаций

In [60]:
# лень разбираться с апи pubchem, но нужно добавить неомицин
Neomycin = {
    'drug': ['Neomycin'],
    'smiles': [
        'C1[C@H]([C@@H]([C@H]([C@@H]([C@H]1N)O[C@@H]2[C@@H]([C@H]([C@@H]([C@H](O2)CN)O)O)N)O[C@H]3[C@@H]([C@@H]([C@H](O3)CO)O[C@@H]4[C@@H]([C@H]([C@@H]([C@@H](O4)CN)O)O)N)O)O)N'
    ]
}
row_to_append = pd.DataFrame(Neomycin)
df_drug = pd.concat([row_to_append, df_drug]).reset_index(drop=True)
df_drug.head(1)

,drug,smiles
0,Neomycin,C1[C@H]([C@@H]([C@H]([C@@H]([C@H]1N)O[C@@H]2[C...


In [63]:
# преведем все к общему виду для облегчения объединения
df_drug = df_drug.rename(columns={'drug': 'Drug', 'smiles': 'Smiles'})
df_drug.head(1)

,Drug,Smiles
0,Neomycin,C1[C@H]([C@@H]([C@H]([C@@H]([C@H]1N)O[C@@H]2[C...


## Adding descriptors

In [66]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

import pandas as pd
from pandas import DataFrame

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors 


def add_desc_to_df(df: pd.DataFrame, desc: list):
    calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc)
    desc_df = pd.DataFrame(
        df['Smiles'].map(lambda x: calc.CalcDescriptors(Chem.MolFromSmiles(x))).to_list()
    )
    desc_df.columns = desc
    return df.join(desc_df)


def fill_nan_desc(df: pd.DataFrame, non_desc_cols: list = None):
    '''
    Fill NaN with calculated descriptors

    Parameters
    ----------
    df: DataFrame

    non_desc_cols: columns from df cant be calculated via rdkit 
    '''
    cols_to_drop = ['Smiles']
    cols_to_drop.extend(non_desc_cols)
    for col in df.drop(columns=cols_to_drop).columns:
        col_type = df[col].dtype
        calc = MoleculeDescriptors.MolecularDescriptorCalculator([col])
        mask = df[col].isna()
        df.loc[mask, col] = df.loc[mask, 'smiles'].map(
            lambda x: calc.CalcDescriptors(Chem.MolFromSmiles(x))[0]
        ).astype(col_type)
    return df

In [67]:
descs_to_add = [
    'LabuteASA', 'NumHDonors', 'NumHAcceptors', 
    'MolLogP', 'Solubility', 'MolWt', 'Chi0'
]
df_drug = add_desc_to_df(df_drug, descs_to_add)

# Мерджим все бд в одну

In [68]:
data_with_bac = pd.merge(df_data, df_bac, left_on='Bacteria', right_on='Bacteria', how='outer')

In [69]:
df = pd.merge(data_with_bac, df_drug, left_on='Drug', right_on='Drug', how='outer')

In [71]:
df.head(1)

,Bacteria,NP_Synthesis,Drug,Drug_class_drug_bank,Drug_dose,NP_concentration,NP size_min,NP size_max,NP size_avg,shape,...,biosafety_level,isolated_from,Smiles,LabuteASA,NumHDonors,NumHAcceptors,MolLogP,Solubility,MolWt,Chi0
0,Escherichia coli,chem_synthesis_reduction_by_D_maltose,Amoxicillin,Lactams,NaN,NaN,4.0,10.0,8.0,spherical,...,2.0,urine,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccc(O)cc3)...,147.914892,4.0,6.0,0.0237,777.0,365.411,18.516143
